In [1]:
from IPython.core.display import display, HTML

def show(string):
    display(HTML(string))
    
def kernel_restart():
    HTML("<script>Jupyter.notebook.kernel.restart()</script>")

# a. Email Support with Flask-Mail

In [1]:
import os
from flask import Flask, render_template, session, redirect, url_for
from flask_bootstrap import Bootstrap
from flask_moment import Moment
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField
from wtforms.validators import DataRequired

# NEW
from flask_sqlalchemy import SQLAlchemy
import sqlite3
import pandas as pd

from flask_mail import Mail




app = Flask(__name__)
app.config['SECRET_KEY'] = 'hard to guess string'
#app.config['SERVER_NAME'] = 'floydjjluo.me'
app.config['SERVER_NAME'] = '127.0.0.1'
app.config['WTF_CSRF_ENABLED'] = False


# basedir = os.path.abspath(os.path.dirname(__file__)); print(basedir)
basedir = ''
DB_URI  = 'sqlite:///' + os.path.join(basedir, 'data.sqlite');print(DB_URI)
app.config['SQLALCHEMY_DATABASE_URI'] = DB_URI
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

bootstrap = Bootstrap(app)
moment = Moment(app)
db = SQLAlchemy(app)

sqlite:///data.sqlite


In [2]:
# Email

'''
app.config['MAIL_SERVER'] = 'smtp.googlemail.com'
app.config['MAIL_PORT'] = 587
app.config['MAIL_USE_TLS'] = True
app.config['MAIL_USERNAME'] = os.environ.get('MAIL_USERNAME')
app.config['MAIL_PASSWORD'] = os.environ.get('MAIL_PASSWORD')
'''

app.config['MAIL_SERVER'] = '220.181.12.16' #'smtp.163.com'
app.config['MAIL_PORT'] = 25
app.config['MAIL_USE_TLS'] = True
app.config['MAIL_USERNAME'] = 'flaskynotebook@163.com'
app.config['MAIL_PASSWORD'] = 'flaskynb211'

In [3]:
mail = Mail(app)

### 1.1 Sending Email from the Python Shell

In [4]:
from flask_mail import Message

# from hello import mail

msg = Message('flasky-notebook-ch6-email-test',
              sender = 'flaskynotebook@163.com',
              recipients = ['flaskynotebook@163.com'])
msg.body = 'This is a plain text body'
msg.html = 'This is a <b>HTML</b> body'

In [5]:
with app.app_context():
    mail.send(msg)

### 1.2 Integrating Emails with the Application

In [13]:
app.config['FLASKY_MAIL_SUBJECT_PREFIX'] = '[Flasky-NB]'
app.config['FLASKY_MAIL_SENDER'] = 'flaskynotebook@163.com'

In [7]:
class Role(db.Model):
    __tablename__ = 'roles'
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(64), unique=True)
    users = db.relationship('User', backref='role', lazy='dynamic')

    def __repr__(self):
        return '<Role %r>' % self.name


class User(db.Model):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(64), unique=True, index=True)
    role_id = db.Column(db.Integer, db.ForeignKey('roles.id'))

    def __repr__(self):
        return '<User %r>' % self.username

In [8]:
%%writefile templates/mail/new_user.html
User <b>{{ user.username }}</b> has joined.


%%writefile templates/mail/new_user.text
User {{ user.username }} has joined.

Overwriting templates/mail/new_user.html


In [9]:
def send_email(to, subject, template, **kwargs):
    
    msg = Message(app.config['FLASKY_MAIL_SUBJECT_PREFIX'] + subject,
                  sender=app.config['FLASKY_MAIL_SENDER'], 
                  recipients=[to])
    msg.body = render_template(template + '.txt', **kwargs)
    
    msg.html = render_template(template + '.txt', **kwargs)
    mail.send(msg)

In [21]:
app.config['FLASKY_ADMIN'] = 'flaskynotebook@163.com'

In [22]:
# fake user
user = User(username = 'FloydLuo')
user.username

'FloydLuo'

In [23]:
with app.app_context():
    send_email(app.config['FLASKY_ADMIN'] ,
               "New User Joined!", 
               'mail/new_user',
               user = user)
    

** Start From Here **

In [1]:
# kernel_restart()
from IPython.core.display import display, HTML

def show(string):
    display(HTML(string))
    
def kernel_restart():
    HTML("<script>Jupyter.notebook.kernel.restart()</script>")
    
    
import os

try:
    os.remove('data.sqlite')
    
except:
    pass

In [2]:
import os
from threading import Thread
from flask import Flask, render_template, session, redirect, url_for
from flask_bootstrap import Bootstrap
from flask_moment import Moment
from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField
from wtforms.validators import DataRequired
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from flask_mail import Mail, Message

basedir = ''

app = Flask(__name__)

app.config['WTF_CSRF_ENABLED'] = False
app.config['SERVER_NAME'] = 'floydjjluo.me'

app.config['SECRET_KEY'] = 'hard to guess string'


app.config['SQLALCHEMY_DATABASE_URI'] ='sqlite:///' + os.path.join(basedir, 'data.sqlite')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

app.config['MAIL_SERVER'] = 'smtp.163.com'
app.config['MAIL_PORT'] = 25
app.config['MAIL_USE_TLS'] = True
app.config['MAIL_USERNAME'] = 'flaskynotebook@163.com'
app.config['MAIL_PASSWORD'] = 'flaskynb211'


app.config['FLASKY_MAIL_SUBJECT_PREFIX'] = '[Flasky-NB]'
app.config['FLASKY_MAIL_SENDER'] = 'flaskynotebook@163.com'
app.config['FLASKY_ADMIN'] = 'flaskynotebook@163.com'

bootstrap = Bootstrap(app)
moment = Moment(app)
db = SQLAlchemy(app)
migrate = Migrate(app, db)
mail = Mail(app)

class Role(db.Model):
    __tablename__ = 'roles'
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(64), unique=True)
    users = db.relationship('User', backref='role', lazy='dynamic')

    def __repr__(self):
        return '<Role %r>' % self.name


class User(db.Model):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(64), unique=True, index=True)
    role_id = db.Column(db.Integer, db.ForeignKey('roles.id'))

    def __repr__(self):
        return '<User %r>' % self.username

db.create_all()


def send_email(to, subject, template, **kwargs):
    
    msg = Message(app.config['FLASKY_MAIL_SUBJECT_PREFIX'] + subject,
                  sender=app.config['FLASKY_MAIL_SENDER'], 
                  recipients=[to])
    msg.body = render_template(template + '.txt', **kwargs)
    
    msg.html = render_template(template + '.txt', **kwargs)
    mail.send(msg)


class NameForm(FlaskForm):
    name = StringField('What is your name?', validators=[DataRequired()])
    submit = SubmitField('Submit')



In [3]:
app_ctx = app.app_context()
app_ctx.push()
client = app.test_client(use_cookies=True)

In [4]:
@app.route('/', methods=['GET', 'POST'])
def index():
    form = NameForm()
    if form.validate_on_submit():
        user = User.query.filter_by(username=form.name.data).first()
        if user is None:
            user = User(username=form.name.data)
            db.session.add(user)
            session['known'] = False
            
            ## NEW ##
            if app.config['FLASKY_ADMIN']:
                send_email(app.config['FLASKY_ADMIN'], 
                           'New User',
                           'mail/new_user', user=user)
            ## NEW ##
        else:
            session['known'] = True
        session['name'] = form.name.data
        form.name.data = ''
        # return redirect(url_for('index'))
        return client.get('/')
    o =   render_template('index.html', 
                           form=form, 
                           name=session.get('name'),
                           known=session.get('known', False)); show(o)
    return o



In [5]:
client.get('/')

<Response streamed [200 OK]>

In [6]:
client.post('/')

<Response streamed [200 OK]>

In [7]:
client.post('/', data = {'name' : 'Faker Man'})

<Response streamed [200 OK]>

In [8]:
client.get('/')

<Response streamed [200 OK]>

In [9]:
client.post('/', data = {'name' : 'New Man'})

<Response streamed [200 OK]>

In [10]:
client.get('/')

<Response streamed [200 OK]>

In [11]:
client.post('/', data = {'name' : 'New Man'})

<Response streamed [200 OK]>

In [12]:
client.get('/')

<Response streamed [200 OK]>

### 1.3 Sending Asynchronous Email


In [14]:
from threading import Thread



def send_async_email(app, msg):
    with app.app_context():
        mail.send(msg)


def send_email(to, subject, template, **kwargs):
    msg = Message(app.config['FLASKY_MAIL_SUBJECT_PREFIX'] + ' ' + subject,
                  sender=app.config['FLASKY_MAIL_SENDER'], 
                  recipients=[to])
    msg.body = render_template(template + '.txt', **kwargs)
    msg.html = render_template(template + '.html', **kwargs)
    thr = Thread(target=send_async_email, args=[app, msg])
    thr.start()
    return thr

# everything else is the same